In [2]:
import pandas as pd
import numpy as np
# use of difflib is when users enter movie by spelling mistake to compare that movie for a given dataset
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
movie_recom= pd.read_csv('C:/Users/sai kumar/Downloads/movies.csv')
movie_recom.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movie_recom.shape

(4803, 24)

In [5]:
movie_recom.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [6]:
#select relevant features for recommendation
select_features=['genres','keywords','tagline','cast','director']
print(select_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [7]:
#replace null values with null strings
for feature in select_features:
    movie_recom[feature]=movie_recom[feature].fillna('')

In [8]:
#combine all 5 selected features
combined_features=movie_recom['genres']+' '+movie_recom['keywords']+' '+movie_recom['tagline']+' '+movie_recom['cast']+' '+movie_recom['director']

In [9]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [10]:
vectorizer=TfidfVectorizer()

In [11]:
feature_vectors=vectorizer.fit_transform(combined_features)

In [12]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 124266 stored elements and shape (4803, 17318)>
  Coords	Values
  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.247137650

Cosine Similarity

In [13]:
#get similarity scores use cosine similarity
similarity=cosine_similarity(feature_vectors)

In [14]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [15]:
print(similarity.shape)

(4803, 4803)


In [16]:
# getting movie name from user
movie_name=input('Enter your favourite movie name: ')

In [17]:
# create list with all movie names given in dataset
list_of_all_titles=movie_recom['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [18]:
# find close match fro movie name given by user
find_close_match=difflib.get_close_matches(movie_name,list_of_all_titles)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [19]:
close_match=find_close_match[0]
print(close_match)

Iron Man


In [20]:
#finding the index of movie with title
index_of_the_movie=movie_recom[movie_recom.title==close_match]['index'].values[0]
print(index_of_the_movie)

68


In [21]:
#get similaritys movies
similarity_score=list(enumerate(similarity[index_of_the_movie]))
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [22]:
len(similarity_score)

4803

In [26]:
#sorting movies based on their similarity score
sorted_similar_movies=sorted(similarity_score,key=lambda x:x[1],reverse=True)
print(sorted_similar_movies)

[(68, np.float64(1.0)), (79, np.float64(0.40890433998005965)), (31, np.float64(0.3146705244947752)), (7, np.float64(0.23944423963486416)), (16, np.float64(0.22704403782296806)), (26, np.float64(0.21566241096831162)), (85, np.float64(0.20615862984665334)), (182, np.float64(0.19573956139611612)), (511, np.float64(0.16702973947860683)), (3623, np.float64(0.1609246088135586)), (64, np.float64(0.1529992413944514)), (203, np.float64(0.1481866794866512)), (174, np.float64(0.1471993120942043)), (4401, np.float64(0.14505971470107848)), (101, np.float64(0.14401677581826292)), (46, np.float64(0.14216268867232232)), (169, np.float64(0.1380947013224906)), (1740, np.float64(0.1362438264169076)), (94, np.float64(0.13616819579029016)), (788, np.float64(0.13305895074229218)), (126, np.float64(0.13263982780511063)), (131, np.float64(0.13137698586006535)), (33, np.float64(0.13089810941050173)), (2487, np.float64(0.12309731939910509)), (783, np.float64(0.12162995562040377)), (138, np.float64(0.11846458075

In [30]:
print('Movies suggested for you: \n')
i=1
for movie in sorted_similar_movies:
    index=movie[0]
    title_from_index=movie_recom[movie_recom.index==index]['title'].values[0]
    if(i<30):
        print(i,'.',title_from_index)

Movies suggested for you: 

1 . Iron Man
1 . Iron Man 2
1 . Iron Man 3
1 . Avengers: Age of Ultron
1 . The Avengers
1 . Captain America: Civil War
1 . Captain America: The Winter Soldier
1 . Ant-Man
1 . X-Men
1 . Made
1 . X-Men: Apocalypse
1 . X2
1 . The Incredible Hulk
1 . The Helix... Loaded
1 . X-Men: First Class
1 . X-Men: Days of Future Past
1 . Captain America: The First Avenger
1 . Kick-Ass 2
1 . Guardians of the Galaxy
1 . Deadpool
1 . Thor: The Dark World
1 . G-Force
1 . X-Men: The Last Stand
1 . Duets
1 . Mortdecai
1 . The Last Airbender
1 . Southland Tales
1 . Zathura: A Space Adventure
1 . Sky Captain and the World of Tomorrow
1 . The Amazing Spider-Man 2
1 . The Good Night
1 . Tropic Thunder
1 . X-Men Origins: Wolverine
1 . Se7en
1 . Zoom
1 . Fantastic Four
1 . Mystery Men
1 . Contagion
1 . Red Sonja
1 . Gothika
1 . Mallrats
1 . The Judge
1 . A Scanner Darkly
1 . March or Die
1 . Man of Steel
1 . Superman II
1 . The Nativity Story
1 . Two Lovers
1 . Charlie Bartlett
1 . Re

In [32]:
movie_name=input('Enter your favourite movie name:')
list_of_all_titles=movie_recom['title'].tolist()
find_close_match=difflib.get_close_matches(movie_name,list_of_all_titles)
close_match=find_close_match[0]
index_of_the_movie=movie_recom[movie_recom.title==close_match]['index'].values[0]
sorted_similar_movies=sorted(similarity_score,key=lambda x:x[1],reverse=True)
similarity_score=list(enumerate(similarity[index_of_the_movie]))
print(similarity)
print('Movies suggested for you: \n')
i=1
for movie in sorted_similar_movies:
    index=movie[0]
    title_from_index=movie_recom[movie_recom.index==index]['title'].values[0]
    if(i<30):
        print(i,'.',title_from_index)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]
Movies suggested for you: 

1 . Iron Man
1 . Iron Man 2
1 . Iron Man 3
1 . Avengers: Age of Ultron
1 . The Avengers
1 . Captain America: Civil War
1 . Captain America: The Winter Soldier
1 . Ant-Man
1 . X-Men
1 . Made
1 . X-Men: Apocalypse
1 . X2
1 . The Incredible Hulk
1 . The Helix... Loaded
1 . X-Men: First Class
1 . X-Men: Days of Future Past
1 . Captain America: The First Avenger
1 . Kick-Ass 2
1 . Guardians of the Galaxy
1 . Deadpool
1 . Thor: The Dark World
1 . G-Force
1 . X-Men: The Last Stand
1 . Duets
1 . Mortdecai
1 . The Last Airbender
1 